# Modelisation d'une epidemie de maladie infectieuse, le modele SIR et la peste d'Eyam

### Cedric Chauve, LaBRI et Simon Fraser University
### cedric.chauve@u-bordeaux.fr, cedric.chauve@sfu.ca
 
Cette activite a pour but d'introduire des notionsd e programmation de base en python et Jupyter, en se basant sur la programmation d'un modele classique d'evolutuion d'une maladie infectieuse dans une population, le modele SIR. Le modele developpe sera applique a l'analyse d'une epidemie de peste bubonique dans le village d'Eyam, en Angleterre au XVIIeme siecle. 
Du point de vue informatique, cette activite permet de manipuler des structures de controle et de donnees simples (boucles, boucles imbriquees, tableaux, dictionnaires) et de developper une petite interface graphique.

In [1]:
# Modules

import math as m
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Widgets & Display modules, etc..
from ipywidgets import widgets as w
from ipywidgets import Button, Layout, widgets
from IPython.display import display, Javascript, Markdown

# grid features for interactive grids 
grid_features = { 'fullWidthRows': True,
                  'syncColumnCellResize': True,
                  'forceFitColumns': True,
                  'rowHeight': 40,
                  'enableColumnReorder': True,
                  'enableTextSelectionOnCells': True,
                  'editable': True,
                  'filterable': False,
                  'sortable': False,
                  'highlightSelectedRow': True}

## Le modele SIR

Le modele SIR est utilise pour simuler et predire l'evolution de la part d'une population donnee infectee par une *maladie infectieuse*, comme la grippe, la peste ou la tuberculose par exemple.

### Description generale du modele

Le modele SIR (pour Susceptible, Infected, Removed, il s'agit donc d'un acronyme anglophone) suppose que la population affectee par une epidemie d'une maladie infectieuse se decompose en trois sous-populations.
- **S.** Les individus *susceptibles* d'etre infectes : ils ne sont pas infectes, mais peuvent le devenir par contact avec un individu infecte.
- **I.** Les individus *infectes* par le pathogene.
- **R.** Les individus *retablis* : ils ne sont ni infectes, ni susceptibles d'etre infectes. Cette partie de la population contient les individus qui ont ete infectes et ne le sont plus soit car ils ont gueri (et sont immunises) soit car ils sont morts.  

Ce modele suppose de plus que la population reste de *taille constante N*, c'est-a-dire que le nombre d'invidus de la population (individus decedes compris) ne change pas au cours de l'epidemie. Il s'agit d'une hypothese qui se justifie dans le cas d'une population dans un environnement ferme (ville en quarantaine par exemple).

Dans le modele SIR, on s'interesse a la *l'evolution de la distribution* des *N* individus de la population au cours d'une periode de temps. On suppose qu'a une date de depart, il y a quelques individus infectes et on se pose la question suivante : apres un laps de temps donne, combien d'individus infectes, susceptibles et retablis observe-t'on dans la population? 

Si l'epidemie en cours suit bien un modele SIR, pouvoir repondre a cette question permet de prendre une decision sur les actions a mener pour contenir l'epidemie. Par exemple, si le modele SIR predit qu'apres un mois il n'y a plus d'individus infectes et une proportion impotante d'individus susceptibles, on peut juger qu'il n'est pas necessaire de faire quoi que ce soit; au contraire, si le modele predit qu'apres un mois une grande partie de la population est infectee, alors on peut decider de mettre en place tres vite des mesures d'isolation des individus infectes pour juguler l'epidemie.

### Description mathematique du modele

On mesure le passage du temps de maniere discrete, on considere donc des intervalles de temps fixes (par exemple une semaine) et on s'interesse au nombre d'individus de chaque type apres une semaine, puis deux semaine, ...

Pour cela on denote par *t* le nombre d'intervalles de temps qui se sont ecolues depuis la decouverte du premier cas d'infection:
- $t=0$ correspond au debut de l'epidemie;
- $t=1$ correspond a l'etat de la population apres un intervalle de temps (dans notre exemple, apres une semaine);
- $t=2$ correspond a l'etat de la population apres deux intervalles de temps (dans notre exemple, apres deux semaines);
- ...

On denote par
- $I(t)$ le nombre d'individus infectes apres $t$ intervalles de temps;
- $S(t)$ le nombre d'individus susceptibles apres $t$ intervalles de temps;
- $R(t)$ le nombre d'individus retablis apres $t$ intervalles de temps.

De par l'hypothese de population constante, on a donc $N=S(t)+I(t)+R(t)$ pour toute valeur entiere non-negative de $t$.

Le but du modele SIR est de pouvoir calculer, a partir du triplet $(S(t),I(t),R(t))$ le triplet $(S(t+1),I(t+1),R(t+1))$, c'est-a-dire de pouvoir predire l'etat de la population un intervalle de temps plus tard.

Ce modele necessite de connaitre deux parametres.
- Le *taux dinfection* $\beta$, qui indique la proportion de contacts entre un individu infecte et un individu susceptible resultant en l'individu susceptible devenant infecte.
- Le *taux de retablissement* $\gamma$ qui indique la proportion d'individus infectes qui se retablissent ou meurent durant un intervalle de temps.

Cela donne le modele mathematique (systeme dynamique) suivant :
$$
\begin{align}
S(t+1) &= S(t) -  {\beta} S(t)I(t)/N\\
I(t+1) &= I(t) +  {\beta} S(t)I(t)/N - \gamma I(t) \\
R(t+1) &= R(t) + \gamma I(t)\\
\end{align}
$$

### Remarques

Le parametre $\gamma$ doit etre compris entre $0$ et $1$. Il est en general aise a determiner : si la maladie prend $\lambda$ intervalles de temps pour suivre son cours chez un individu, alors $\gamma=1/\lambda$. Le modele SIR syppose implicitement que tout individu infecte le restera pour une periode de $1/\gamma$ intervalles de temps, i.e. ne se retablira pas plus vite.

Le parametre $\beta$ est difficile a obtenir au debut d'une epidemie, mais on en connait de bonnes approximations pour la plupart des maladies infectieuses courantes.

## Activite 1.

Ecrire une fonction *discrete_SIR* en python qui prend en entree
- S0,I0,R0 = $S(0),I(0),R(0)$,
- beta (parametre $\beta$),
- gamma (parametre $\gamma$),
- t_max, nombre d'intervalles de temps de l'epidemie
et calcule trois tableaux (dictionaires) S, I, R de t_max+1 entrees chacun tels que S[t]=$S(t)$, I[t]=$I(t)$, R[t]=$R(t)$.

### Visualisation

Ecrire une fonction *plot_SIR* qui prend en entree
- S,I,R = trois dictionnaires modelisant l'evolution d'une epidemie,
- size1, size2 = taille d'un graphique,
- time = description de l'unite de temps,
- title = titre de la figure,
et graphe l'evolution d'une epidemie decrite par les tableaux S,I,R.

### Application

Appliquer les deux fonctions precedentes a l'analyse des epidemies suivantes :
- grippe, $\beta=0.6,\ \lambda = 6$ (la maladie dure 6 jours), t_max = 90;
- grippe peu virulente, $\beta=0.15,\ \lambda = 6$ (6 jours), t_max = 90;
- rougeole, $\beta=0.8,\ \lambda = 20$ (20 jours), t_max = 90;
- grippe de Hong-Kong (1968-1969), $\beta=0.5,\ \lambda = 3$ (3 jours), t_max = 90;
- peste de Bombay (1905-1906), $\beta=0.0273,\ \lambda = 56.2$ (56.2 jours), t_max = 600.  

Dans tous les cas on suppose que $N=1000, S(0)=990$ et $I(0)=10$.

## La peste d'Eyam

On suppose maintenant que l'on dispose des donnees d'une population victime d'une epidemie, dont on fait l'hypothese qu'elle evolue plus ou moins selon un model SIR, dont on veut estimer les parametres $\beta$ et $\gamma$ que l'on ne connait a priori pas. Le parametre $\gamma$ peut etre obtenu assez facilement en faisant la moyenne du temps durant lequel chaque individu infecte reste malade avant soit de mourir soit de se retablir. On veut donc estimer le parametre $\beta$ a partir de donnees que l'on a pour pouvoir prevoir la dynamique de l'epidemie. Comment proceder pour choisir le meilleur parametre $\beta$. 

Comme application, on prend une peidemie de peste bubonique dans le village anglais d'Eyam, en 1665, annee de la Grande Peste de Londres, une des dernieres epidemies mqjeuresde la seconde pandemie de peste qui a frappe l'Europe a la suite de la Peste Noire du XIVeme siecle et jusqu'au XVIIIeme siecle (Peste de Marseilles en 1720).

| Date de collection des donnees  |Nombre de jours depuis le debut de l'epidemie |Susceptibles | Infectes | 
|-------||-------------|----------|
|June 19 1665|0|254|7| 
|July 3 1665|14|235|14|
|July 19 1665|28|201|22|
|Aug 3 1665|42|153|29|
|Aug 19 1665|56|121| 21|
|Sept 3 1665|70|108|8|
|Sept 19 1665|84|97|21|
|Oct 3 1665| 98|NA | NA|
|Oct 19 1665|112| 83 | 0|

Cette epidemie est interessante car des qu'elle fut detectee, le village d'Eyam s'est ferme, empechant toute entree ou sortie, jusqu'a la fin de l'epidemie. L'hypothese de population constante du modele SIR est donc satisfaite et on peut se poser la question suivante : l'epidemie d'Eyam suit-elle un modele SIR et si oui, pour quelles valeurs de $\beta$ et $\gamma$? Pour $\gamma$ on peut utiliser l'observation que les patients infectes le restent entre 10 et 14 jours (donc $10 \leq \lambda \leq 14$). Pour $\beta$ on ne dispose que des donnees numeriques ci-dessus.

## Activite 2

A partir des informations fournies precedemment, ecrire une fonction *parameters_SIR* qui prend en entree
- S,I,R = trois dictionnaires modelisant l'evolution d'une epidemie,
- t_max = la duree de  l'epidemie
- lambda_min, lambda_max : intervalle des valeurs possibles de $\lambda$ (deux entiers),
- beta_min, beta_max : intervalle des valeurs possibles de $\beta$ (deux reels).
et calcule la valeur des parametres $\beta$ et $\gamma$ d'un modele SIR qui modelise les donnees fournies le plus precisement possible.

### Application
Appliquons notre fonction aux donnees d'Eyam.

On voit que les parametres obtenus par notre recherche menent a un modele qui cadre assez bien avec les donnees, malgre un decalage du croisement de la courbe des susceptibles et des retablis.

## Une interface graphique pour le modele SIR

Nous allons maintenant developper une interface graphique pour visualiser un model SIR de type Modele-Vue-Controleur. 